In [133]:
import numpy as np
import os
import matplotlib
from matplotlib import pyplot as plt
from scipy import signal

GRAY_CONVERTER = np.array([0.2989, 0.5870, 0.1140])

train_path = r'.\Group_1\train'
test_path = r'.\Group_1\test'


In [134]:
def relu(x):
    return np.where(x >= 0, x, 0)


def sigmoid(x):
    return np.apply_along_axis(lambda a: 1 / (1 + np.exp(-a)), 0, x)


def convert_to_gray(image):
    return np.dot(image[..., :3], GRAY_CONVERTER)


In [135]:
class ConvolutionLayer:
    def __init__(self, F, K):
        self.F = F
        self.K = K
        self.filters = np.random.normal(scale=0.3, size=(K, F, F))

    def convolve(self, input):
        H, W = input.shape

        for i in range(H - self.F + 1):
            for j in range(W - self.F + 1):
                section = input[i: i + self.F, j: j + self.F]
                yield section, i, j

    def calculate(self, input):
        H, W = input.shape

        feature_maps = np.zeros((H - self.F + 1, W - self.F + 1, self.K))

        for section, i, j in self.convolve(input):
            feature_maps[i, j] = np.sum(section * self.filters, axis=(1, 2))

        return feature_maps


In [136]:
conv_layer1 = ConvolutionLayer(3, 32)
conv_layer2 = ConvolutionLayer(3, 64)


In [137]:
image = matplotlib.image.imread(os.path.join(train_path, 'bird', '0000.jpg'))


In [138]:
grayscale_image = convert_to_gray(image)


In [139]:
output_layer1 = conv_layer1.calculate(grayscale_image)


In [140]:
activated_output_layer1 = relu(output_layer1)